In [1]:
!pip install nltk

In [2]:
import torch.nn as nn
import torch
import torch.nn.utils.rnn as rnn
import statistics
import nltk # 없으시면 설치하세요: pip install nltk
import random
import collections
import time

In [3]:
#gpu 디바이스 설정

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
## Dictionary class 선언 (사전 만드는 부분)
class Dictionary(object):
    def __init__(self, dataset, size):
        ## init vocab ##
        self.word2idx = {'<pad>':0, '<sos>': 1, '<eos>': 2, '<unk>': 3} # 사전 
        self.idx2word = ['<pad>', '<sos>', '<eos>', '<unk>'] # inverted dictionary
        #sos는 시작, eos는 끝, unk는 사전에 없는 단어가 등장할 경우 unk에 대한 인덱스 부여
        #self.word2idx['unk'] -> 3
        #self.idx2word[3] -> unk

        self.build_dict(dataset, size)
        #dataset은 데이터, size는 사전 크기(1만으로 할 지, 2만으로 할 지, 하이퍼 파라미터 (너무 높아도 문제, 학습해야 할 부분이 많아서) )

    def __call__(self, word):
        return self.word2idx.get(word, self.word2idx['<unk>']) # if word does not exist in vocab then return unk idx
    #키가 있으면 키에 대한 값을 출력, 없으면 unk의 값을 출력

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def build_dict(self, dataset, dict_size): #밑에서 가져온 문장들을 통해 사전 만드는 법 (가장 많이 등장한 단어 순서로 1,2,3..)
        ## Practice ##
        """Tokenize a text file."""
        total_words = (word for sent in dataset for word in sent) # store all words into tuple (단어 등장 빈도 수를 알아내고)
        word_freq = collections.Counter(total_words)# count the number of each word: ex) ('The': 10000, 'a': 5555, ...) (그걸 튜플로 묶어냄)
        vocab = sorted(word_freq.keys(), key=lambda word: (-word_freq[word], word)) # sort by frequency
        #
        vocab = vocab[:dict_size] # truncate
        for word in vocab:
            self.add_word(word)

    def __len__(self):
        return len(self.idx2word)


In [5]:
## Brown dataset Preprocessing (NLTK)
def brown_dataset(min=5, max=30): #문장 길이가 min~max 사이인 것만 받고, 모든 문장을 소문자로 치환
    nltk.download('brown')

    # get sentences with the length between min and max
    # convert all words into lower-case
    all_seq = [[token.lower() for token in seq] for seq in nltk.corpus.brown.sents() 
               if min <= len(seq) <= max]

    random.shuffle(all_seq) # shuffle
    return all_seq

In [6]:
## Download Brown dataset
dataset = brown_dataset()
print(len(dataset)) #문장이 몇 개인가(43450개는 사실 작은 데이터양, 100만개 단위가 되어도 모자란 경우 있을 수)
## print some part
print(dataset[0]) #한 문장씩 저장되어 있는 것을 알 수 있음
print(dataset[1])
print(dataset[2])

[nltk_data] Downloading package brown to /home/piai/nltk_data...
[nltk_data]   Package brown is already up-to-date!


43450
['we', 'followed', 'the', 'asphalt', 'road', 'for', 'a', 'few', 'miles', 'and', 'then', 'swung', 'off', 'onto', 'a', 'smaller', 'road', 'which', 'was', 'nothing', 'more', 'than', 'two', 'tire', 'marks', 'on', 'the', 'earth', '.']
['press', 'scored', 'handle', 'ends', 'firmly', 'in', 'place', 'using', 'dowel', 'to', 'reinforce', 'container', 'while', 'pressing', ';', ';']
['you', 'know', 'that', 'i', 'could', 'hold', 'right', 'here', 'in', 'my', 'hand', 'the', 'little', 'chunk', 'of', 'uranium', 'metal', 'that', 'was', 'the', 'heart', 'of', 'the', 'bomb', 'that', 'dropped', 'on', 'hiroshima', '.']


In [7]:
## Data handler class 선언
class Corpus(object):
    def __init__(self, dataset, device, dict_size=20000, train_ratio=0.97):
        #43450개의 데이터 중 97%를 학습용으로 쓰고, 나머지 3%를 test로 사용하겠다
        
        train_size = int(len(dataset) * train_ratio)
        self.device = device
        self.dictionary = Dictionary(dataset, dict_size) #위의 딕셔너리 클래스
        self.train = dataset[:train_size] # [0 ~ train_size]
        self.valid = dataset[train_size:] # [train_size: len(dataset)]

    def indexing(self, dat):
        # dat = list(list)
        
        src_idxes = [] # 모델 입력
        tgt_idxes = [] # 모델 정답
        for sent in dat:
            #torch.tensor은 리스트로 받아야 하므로,
            src_idx = [self.dictionary('<sos>')] + [self.dictionary(word) for word in sent] #입력받은 애들을
            tgt_idx = [self.dictionary(word) for word in sent] + [self.dictionary('<eos>')]
            src_idxes.append(torch.tensor(src_idx).type(torch.int64)) 
            tgt_idxes.append(torch.tensor(tgt_idx).type(torch.int64))
        
        #batch_first를 true로 두면, B*L로 둔다. (즉, 칸이 모자란 짧은 문장에 batch해줌)
        #view(-1): 미리 플랫하게 만들어둠, 나중에 flat하게 가야 하므로
        src_idxes = rnn.pad_sequence(src_idxes, batch_first=True).to(self.device) # shape = [B, L]
        tgt_idxes = rnn.pad_sequence(tgt_idxes, batch_first=True).to(self.device).view(-1) # flatten shape = [B * L]

        return src_idxes, tgt_idxes

    def batch_iter(self, batch_size, isTrain=True):
        dat = self.train if isTrain else self.valid
        if isTrain:
            random.shuffle(dat)

        for i in range(len(dat) // batch_size):
            batch = dat[i * batch_size: (i+1) * batch_size]
            src, tgt = self.indexing(batch)
            yield {'src': src, 'tgt': tgt}
            #트레이닝 할 때 출력함

In [8]:
corpus = Corpus(dataset, device)

In [9]:
# Dictionary 확인
for i, (key, val) in enumerate(corpus.dictionary.word2idx.items()):
    print('word:  {:10s} | index: {:5d} '.format(key, val))
    if i == 20:
        break

word:  <pad>      | index:     0 
word:  <sos>      | index:     1 
word:  <eos>      | index:     2 
word:  <unk>      | index:     3 
word:  the        | index:     4 
word:  .          | index:     5 
word:  ,          | index:     6 
word:  of         | index:     7 
word:  and        | index:     8 
word:  to         | index:     9 
word:  a          | index:    10 
word:  in         | index:    11 
word:  was        | index:    12 
word:  he         | index:    13 
word:  is         | index:    14 
word:  ''         | index:    15 
word:  ``         | index:    16 
word:  it         | index:    17 
word:  that       | index:    18 
word:  for        | index:    19 
word:  ;          | index:    20 


In [10]:
## indexing 함수 결과 확인 (문장 하나만 입력했다고 했을 떄 확인)

# case : 단일 문장 입력 시. 
sent = [dataset[1]]
idx_src, idx_tgt = corpus.indexing(sent)

print(sent)
print(idx_src) # <SOS> index로 시작
print(idx_tgt) # <EOS> index로 종료

print('-' * 90)
## case : 복수 문장 입력 시 (batching)
batch = [dataset[0], dataset[1]]
idx_src, idx_tgt = corpus.indexing(batch)

print(batch)
print(idx_src) # 가장 길이가 긴 문장 (dataset[0]) 보다 짧은 문장 (dataset[1]) 의 경우 남는 길이만큼 padding=0 삽입 확인.
print(idx_tgt)

[['press', 'scored', 'handle', 'ends', 'firmly', 'in', 'place', 'using', 'dowel', 'to', 'reinforce', 'container', 'while', 'pressing', ';', ';']]
tensor([[    1,   807,  4764,  2048,  1395,  2490,    11,   174,   665, 17037,
             9,  7694,  6083,   188,  3818,    20,    20]], device='cuda:0')
tensor([  807,  4764,  2048,  1395,  2490,    11,   174,   665, 17037,     9,
         7694,  6083,   188,  3818,    20,    20,     2], device='cuda:0')
------------------------------------------------------------------------------------------
[['we', 'followed', 'the', 'asphalt', 'road', 'for', 'a', 'few', 'miles', 'and', 'then', 'swung', 'off', 'onto', 'a', 'smaller', 'road', 'which', 'was', 'nothing', 'more', 'than', 'two', 'tire', 'marks', 'on', 'the', 'earth', '.'], ['press', 'scored', 'handle', 'ends', 'firmly', 'in', 'place', 'using', 'dowel', 'to', 'reinforce', 'container', 'while', 'pressing', ';', ';']]
tensor([[    1,    48,   516,     4, 15767,   502,    19,    10,   173,   578

In [11]:
## RNN Language model 선언

# Define network
class RNNModel(nn.Module):
    #init 부분에서는 학습할 weight, layer등을 정의하는 곳
    #forward 부분에서는 init에서 저장된 weight를 이용해서 연산을 정의하는 곳
    
    def __init__(self, ntoken, hidden_size, nlayers, dropout=0.1):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout) #그냥 드랍아웃
        self.embeddings = nn.Embedding(ntoken, hidden_size, padding_idx=0) #(딕셔너리크기, 차원의 크기)
        self.rnn = nn.LSTM(hidden_size, hidden_size, nlayers, dropout=dropout, batch_first=True) #(입력차원의 크기, 나왔을 때 차원크기)
        #입력 했을 때 차원 크기이므로, self.embedding 했을 때 나오는 차원의 크기와 같아야 함. 
        #나왔을 때 차원 크기는 그냥 임의로 같게 해줌
        
        self.output_layer = nn.Linear(hidden_size, ntoken) # 입력 차원의 크기, 나왔을 때 차원의 크기
        #나왔을 때 차원의 크기는 반드시 사전의 크기와 같아야 하고, 입력 차원은 앞의 nn.LSTM에서의 나올 때 차원의 크기와 같아야 함
        self.sm = nn.LogSoftmax(dim=-1) # log확률값(다시 로그 씌우지 않아도 됨, 여기서 로그 계산 됨, 그냥 더하기만 하면 됨)

        self.ntoken = ntoken
        self.hidden_size = hidden_size
        self.nlayers = nlayers

        self.init_weights()

    def init_weights(self): #모델 웨이트 초기화 (안중요함)
        initrange = 0.1
        self.embeddings.weight.data.uniform_(-initrange, initrange)
        self.output_layer.weight.data.uniform_(-initrange, initrange)
        self.output_layer.bias.data.zero_()

    def forward(self, input, hidden):#학습 할 때 호출되는 부분(모델 생성에 중요)
        # shape(input) = [Batch, length]
        emb = self.embeddings(input) # emb = (batch, length, dim) > 3차원화
        #input한 word들을 넣고, 그걸 embedding 한 것을 밑에 넣으면, 앞에서 ht-n과 자신의 것을 합쳐서 연결해서 구하는 것을 자동으로 해줌
        output, hidden = self.rnn(emb) # output = (batch. length. dim) #output은 ht-n,..,ht-1까지, 히든은 마지막 것에 대한 h
        output = self.drop(output) #dropout 
        output = self.output_layer(output) # linear projection : hidden dim --> vocab size
        #마지막 소프트맥스 레이어에 들어가기 전에는 voca (사전)사이즈로 맞춰줘야 한다.
        output = output.view(-1, self.ntoken) # output = (batch * length, vocab_size)
        output = self.sm(output)# softmax

        return output, hidden

    def init_hidden(self, bsz): #첫번째 애도 앞에서 받는 게 있어야 하니까, 앞에 아무것도 없는 벡터가 있다는 것 같은 걸 만들어 줌
        weight = next(self.parameters()) # to set init tensor with the same torch.dtype and torch.device
        return (weight.new_zeros(self.nlayers, bsz, self.hidden_size),
                weight.new_zeros(self.nlayers, bsz, self.hidden_size)) #H1 과 셀에 대해서 제로 벡터를 받으려구 두 개가 있음


In [12]:
# Hyperparameters (설정할 하이퍼파라미터)
batch_size = 60
hidden_size = 256
dropout = 0.2
max_epoch = 30

# build model
ntokens = len(corpus.dictionary) #사전의 크기
model = RNNModel(ntokens, hidden_size, 1, dropout).to(device) #RNN 클래스 호출
isTrain=True # Flag variable
#TRUE로 두어야 모델이 학습됨, FALSE로 둔다는 건 처음 학습 한 걸로 계속 돌아감.(보통 계속 학습할 이유가 없으니까)

# set loss func and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.NLLLoss(ignore_index=0, reduction='mean')

/home/piai/anaconda3/envs/AIBD/lib/python3.8/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [13]:
##### Training / Evaluation Parts #######

In [14]:
# accuracy (학습 중간중간 정확도 계산부분)
def cal_acc(scores, target):
    pred = scores.max(-1)[1]
    non_pad = target.ne(0)
    num_correct = pred.eq(target).masked_select(non_pad).sum().item() 
    num_non_pad = non_pad.sum().item()
    return 100 * (num_correct / num_non_pad)

In [15]:
# train func.
# corpus부분 실행, forward 실행 부분

def train():
    model.train() # Turn on training mode which enables dropout.
    mean_loss = []
    mean_acc = []
    start_time = time.time()

    for batch in corpus.batch_iter(batch_size):
        hidden = model.init_hidden(batch_size) # zero vectors for init hidden
        target = batch['tgt'] # flattened target 
        optimizer.zero_grad()
        output, hidden = model(batch['src'], hidden) # output = flatten output = [Batch_size * Length, vocab_size]
        #이 때, 아웃풋은 각각의 voca 분포로 나옴
        # output shape = (batch * length, vocab_size)
        # target shape = (batch * length)   --> (batch * length, vocab_size) 로 one-hot distribtuion으로 내부적으로 변환되어 비교 수행
        loss = criterion(output, target) # compare between vocab_prob and answer_prob(one-hot converted)
        loss.backward()
        optimizer.step()

        mean_loss.append(loss.item())
        mean_acc.append(cal_acc(output, target))

    total_time = time.time() - start_time
    mean_acc = statistics.mean(mean_acc)
    mean_loss = statistics.mean(mean_loss)

    return mean_loss, total_time, mean_acc

In [16]:
# evaluation func.
# train과 동일하나, 여기서 학습이 일어나지 않으므로, 학습 부분들은 필요없다.
def evaluate():
    model.eval() # Turn off dropout
    mean_loss = []
    mean_acc = []

    for batch in corpus.batch_iter(batch_size, isTrain=False):
        with torch.no_grad():
            hidden = model.init_hidden(batch_size)
            target = batch['tgt']
            output, hidden = model(batch['src'], hidden)
            loss = criterion(output, target)
            mean_loss.append(loss.item())
            mean_acc.append(cal_acc(output, target))

    mean_acc = statistics.mean(mean_acc)
    mean_loss = statistics.mean(mean_loss)

    return mean_loss, mean_acc

In [17]:
if isTrain: # set False if you don't need to train model
    start_time = time.time()

    for epoch in range(1, max_epoch+1):
        loss, epoch_time, accuracy = train()
        print('epoch {:4d} | times {:3.3f} |  loss: {:3.3f} | accuracy: {:3.2f}'.format(epoch+1, epoch_time, loss, accuracy))

        if epoch % 10 == 0:
            loss, accuracy = evaluate()
            print('=' * 60)
            print('Evaluation | loss: {:3.3f} | accuracy: {:3.2f}'.format(loss, accuracy))
            print('=' * 60)

    with open('model.pt', 'wb') as f:
        print('save model at: ./model.pt')
        torch.save(model, f)

epoch    2 | times 16.601 |  loss: 5.688 | accuracy: 20.46
epoch    3 | times 17.045 |  loss: 5.065 | accuracy: 23.93
epoch    4 | times 17.660 |  loss: 4.710 | accuracy: 25.59
epoch    5 | times 17.654 |  loss: 4.396 | accuracy: 27.11
epoch    6 | times 17.697 |  loss: 4.110 | accuracy: 28.81
epoch    7 | times 16.810 |  loss: 3.861 | accuracy: 30.74
epoch    8 | times 17.214 |  loss: 3.653 | accuracy: 32.66
epoch    9 | times 15.238 |  loss: 3.483 | accuracy: 34.43
epoch   10 | times 16.889 |  loss: 3.340 | accuracy: 36.02
epoch   11 | times 16.581 |  loss: 3.226 | accuracy: 37.44
Evaluation | loss: 5.899 | accuracy: 22.96
epoch   12 | times 17.798 |  loss: 3.128 | accuracy: 38.62
epoch   13 | times 17.332 |  loss: 3.050 | accuracy: 39.59
epoch   14 | times 17.421 |  loss: 2.987 | accuracy: 40.37
epoch   15 | times 17.409 |  loss: 2.930 | accuracy: 41.15
epoch   16 | times 16.840 |  loss: 2.885 | accuracy: 41.69
epoch   17 | times 16.833 |  loss: 2.849 | accuracy: 42.16
epoch   18 | 

In [18]:
def pred_sent_prob(sent):
    model.eval()
    with torch.no_grad():
        # 1. 모델 입력 및 정답 문장에 대한 단어 indexing
        idx_src, idx_tgt = corpus.indexing(sent)
        # 2. initial hidden 생성
        hidden = model.init_hidden(len(sent))
        # 3. LM의 결과(확률분포) 생성
        output, hidden = model(idx_src, hidden)
        # 4. 모델 확률분포로부터 정답 단어의 각 index에 대한 Log 확률 값 추출.
        log_prob = []
        for i in range(len(output)) :
            log_prob.append(output[i][idx_tgt[i]])
        # 5. log 확률의 합.
        sent_prob = sum(log_prob)
        # 6. 결과 return (return type: float)
        return sent_prob

In [19]:
# load saved model
with open('./model.pt', 'rb') as f:
    print('load model from: ./model.pt')
    model = torch.load(f).to(device)

    print('log prob of [the dog bark .]: {:3.3f}'.format(pred_sent_prob([['the', 'dog', 'bark', '.']])))
    print('log prob of [the cat bark .]: {:3.3f}'.format(pred_sent_prob([['the', 'cat', 'bark', '.']])))

    print('log prob of [boy am a i .]: {:3.3f}'.format(pred_sent_prob([['boy', 'am', 'a', 'i', '.']])))
    print('log prob of [i am a boy .]: {:3.3f}'.format(pred_sent_prob([['i', 'am', 'a', 'boy', '.']])))


load model from: ./model.pt
log prob of [the dog bark .]: -39.488
log prob of [the cat bark .]: -53.549
log prob of [boy am a i .]: -47.778
log prob of [i am a boy .]: -20.623


In [20]:
def pred_next_word(partial_sent, topN=3):
    model.eval()
    with torch.no_grad():
        # 1. 모델 입력 및 정답 문장에 대한 단어 indexing
        idx_src, idx_tgt = corpus.indexing(partial_sent)
        # 2. initial hidden 생성
        hidden = model.init_hidden(len(partial_sent))
        # 3. LM의 결과(확률분포) 생성
        output, hidden = model(idx_src, hidden)
        # 4. topN에 해당하는 다음단어의 word index 추출 (Hint: torch.topk() 활용)
        values, idx_word = output.topk(topN)
        # 5. word index --> word 로 변환
        word_list = []
        for i in range(topN) :
            idx_category= idx_word[0][i].item()
            word_list.append(corpus.dictionary.idx2word[idx_category])
       
        # 6. topN word list 반환 (return type: list)
        return word_list

In [21]:
partial_sent = [['the', 'next', 'word']]
N=3
candidates = pred_next_word(partial_sent, topN=N)

# print 
partial_sent = ' '.join(partial_sent[0])
print('Top {0} next words for a partial sentence [{1}] is: '.format(N, partial_sent))
print('===>', candidates)

Top 3 next words for a partial sentence [the next word] is: 
===> ['the', '``', 'he']
